In [1]:
import os
os.chdir('../')
from solution_debugging import *

In [2]:
data_dir = os.curdir
output_dir = os.curdir
raw_train_data = np.load(os.path.join(data_dir, 'train_data.npz'))
x_train = torch.from_numpy(raw_train_data['train_x']).reshape([-1, 784])
y_train = torch.from_numpy(raw_train_data['train_y']).long()
dataset_train = torch.utils.data.TensorDataset(x_train, y_train)

In [3]:
x_train_mini = x_train[:2000]
y_train_mini = y_train[:2000]
dataset_train_mini = torch.utils.data.TensorDataset(x_train, y_train)
model = Model(use_densenet=False, num_epochs=1)
torch.autograd.set_detect_anomaly(True)

Using a BayesNet model
initialising bayesian layer of size 784 x 100
initialising bayesian layer of size 100 x 100
initialising bayesian layer of size 100 x 10


In [4]:
# model.train(dataset_train_mini)

In [5]:
bl = model.network.layers[0]
print(bl)

BayesianLayer(
  (weights_prior): MultivariateDiagonalGaussian()
  (bias_prior): MultivariateDiagonalGaussian()
  (weights_var_posterior): MultivariateDiagonalGaussian()
  (bias_var_posterior): MultivariateDiagonalGaussian()
)


In [6]:
in_features = 28*28
out_features = 10
hidden_features = (100,100)
feature_sizes = (in_features,) + hidden_features + (out_features,)
num_affine_maps = len(feature_sizes) - 1


"""We recreate the network but using softplus=False - for some reason this doesn't lead us to the second time gradient
calculation errors (saying that )

UserWarning: Error detected in SoftplusBackward. Traceback of forward call that caused the error:
  File "C:/Users/benja/Google Drive/eth_courses/pai/task2_bnn/solution_debugging.py", line 629, in <module>
  
  ...
  
      self.bias_var_posterior = MultivariateDiagonalGaussian(torch.nn.Parameter(torch.zeros((out_features,))),torch.nn.Parameter(torch.ones((out_features,))), softplus=softplus)
  File "C:/Users/benja/Google Drive/eth_courses/pai/task2_bnn/solution_debugging.py", line 407, in __init__
    normal = torch.distributions.Normal(self.mu, F.softplus(self.rho))
 (Triggered internally at  ..\torch\csrc\autograd\python_anomaly_mode.cpp:104.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
Traceback (most recent call last):

...

File "C:\Users\benja\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 149, in backward
    allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag
RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have
already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad().
Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved
variables after calling backward.


NBNB!!!

retain_graph=True doesn't actually fix things, gives different error:
RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation:
[torch.FloatTensor [10]] is at version 1; expected version 0 instead. Hint: the backtrace further above shows
the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!



"""
model.network.layers = nn.ModuleList([
            BayesianLayer(feature_sizes[idx], feature_sizes[idx + 1], bias=True, softplus=False)
            for idx in range(num_affine_maps)
        ])

initialising bayesian layer of size 784 x 100
initialising bayesian layer of size 100 x 100
initialising bayesian layer of size 100 x 10


In [7]:

batch_x = x_train[:3]
batch_y = y_train[:3]

# out, log_prior, log_var_ptior = bl.forward(x_batch)
optimizer = torch.optim.Adam(model.network.parameters(), lr=1e-3)
model.network.train()


BayesNet(
  (layers): ModuleList(
    (0): BayesianLayer(
      (weights_prior): MultivariateDiagonalGaussian()
      (bias_prior): MultivariateDiagonalGaussian()
      (weights_var_posterior): MultivariateDiagonalGaussian()
      (bias_var_posterior): MultivariateDiagonalGaussian()
    )
    (1): BayesianLayer(
      (weights_prior): MultivariateDiagonalGaussian()
      (bias_prior): MultivariateDiagonalGaussian()
      (weights_var_posterior): MultivariateDiagonalGaussian()
      (bias_var_posterior): MultivariateDiagonalGaussian()
    )
    (2): BayesianLayer(
      (weights_prior): MultivariateDiagonalGaussian()
      (bias_prior): MultivariateDiagonalGaussian()
      (weights_var_posterior): MultivariateDiagonalGaussian()
      (bias_var_posterior): MultivariateDiagonalGaussian()
    )
  )
  (activation): ReLU()
)

In [8]:
model.network.zero_grad()

out, log_prior, log_var_ptior = model.network.forward(batch_x)
f = out, log_prior, log_var_ptior

class_predicted_probs = F.softmax(f[0], dim=1)
data_predicted_probs = torch.sum(class_predicted_probs * F.one_hot(batch_y, num_classes=10), dim=1)
eps=1e-5
data_predicted_probs = torch.max(data_predicted_probs, torch.tensor(eps).expand_as(data_predicted_probs))
num_not_maxed = torch.sum(data_predicted_probs > eps)
log_prob_of_data_given_network = torch.sum(
    torch.log( data_predicted_probs)
)
# loss = pi_i * (torch.sum(f[2]) - torch.sum(f[1])) - log_prob_of_data_given_network
loss = (torch.sum(f[2]) - torch.sum(f[1])) - log_prob_of_data_given_network

In [9]:
# loss.backward(retain_graph=True)
loss.backward()

In [10]:
optimizer.step()

In [11]:
model.network.layers[-1].weights_var_posterior.rho

Parameter containing:
tensor([1.0010, 0.9990, 0.9990, 1.0010, 1.0010, 1.0010, 0.9990, 1.0010, 1.0010,
        1.0010, 1.0010, 0.9990, 1.0010, 0.9990, 1.0010, 0.9990, 1.0010, 0.9990,
        0.9990, 1.0010, 1.0010, 1.0010, 0.9990, 0.9990, 0.9990, 1.0010, 0.9990,
        1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010,
        1.0010, 0.9990, 1.0010, 0.9990, 0.9990, 1.0010, 1.0010, 0.9990, 0.9990,
        1.0010, 0.9990, 1.0010, 1.0010, 0.9990, 1.0010, 1.0010, 1.0010, 1.0010,
        1.0010, 0.9990, 0.9990, 0.9990, 0.9990, 1.0010, 0.9990, 0.9990, 1.0010,
        1.0010, 1.0010, 0.9990, 1.0010, 0.9990, 0.9990, 1.0010, 1.0010, 1.0010,
        0.9990, 1.0010, 0.9990, 0.9990, 1.0010, 0.9990, 1.0010, 0.9990, 1.0010,
        0.9990, 0.9990, 1.0010, 0.9990, 1.0010, 0.9990, 1.0010, 1.0010, 1.0010,
        1.0010, 1.0010, 0.9990, 1.0010, 1.0010, 1.0010, 0.9990, 1.0010, 1.0010,
        1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010, 1.0010,
        0.9990, 1.

In [12]:

batch_x = x_train[3:6]
batch_y = y_train[3:6]


model.network.zero_grad()

out, log_prior, log_var_ptior = model.network.forward(batch_x)
f = out, log_prior, log_var_ptior

class_predicted_probs = F.softmax(f[0], dim=1)
data_predicted_probs = torch.sum(class_predicted_probs * F.one_hot(batch_y, num_classes=10), dim=1)
eps=1e-5
data_predicted_probs = torch.max(data_predicted_probs, torch.tensor(eps).expand_as(data_predicted_probs))
num_not_maxed = torch.sum(data_predicted_probs > eps)
log_prob_of_data_given_network = torch.sum(
    torch.log( data_predicted_probs)
)
# loss = pi_i * (torch.sum(f[2]) - torch.sum(f[1])) - log_prob_of_data_given_network
loss = (torch.sum(f[2]) - torch.sum(f[1])) - log_prob_of_data_given_network


In [13]:
loss.backward()

In [14]:
optimizer.step()

In [23]:
bl.weights_var_posterior.mu.grad

tensor([ 0.5452,  0.7160, -0.5759,  ..., -1.4657,  0.2981,  0.6082])

In [22]:
model.network.layers[0].weights_var_posterior.mu

Parameter containing:
tensor([0., 0., 0.,  ..., 0., 0., 0.], requires_grad=True)